# Extract player info

In [1]:
from bs4 import BeautifulSoup
import urllib.request as url
import re

def find_player_id_per_year(years):
    players_season = {}
    for year in years:
        players = {}
        url_year = "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/year/" + str(year) + "/qualified/false"
        for page in [-1] + list(range(41, 482, 40)):
            url_page = ""
            if page == -1:
                url_page = url_year
            else:
                url_page = url_year + "/count/" + str(page)
            html_page = url.urlopen(url_page)
            soup = BeautifulSoup(html_page)
            for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/player/_/id")}):
                players_id = link.get('href').split('/')
                players[players_id[len(players_id)-1]] = players_id[len(players_id)-2]
        players_season[str(year-1) + "-" + str(year)] = players
    return players_season;    

In [2]:
import pandas
def read_players(player_id_years, years):
    player_info = {}
    for year in years:
        player_id = player_id_years[str(year-1)+"-"+str(year)]
        for player in player_id:
            url_path = "http://www.espn.com/nba/player/gamelog/_/id/" + player_id[player] + "/year/" + str(year) + "/" + player
            try:
                dataframe = pandas.read_html(url_path)
                if player not in player_info:
                    player_info[player] = {}
                player_info[player][str(year-1)+"-"+str(year)] = dataframe
            except url.HTTPError:
                pass
    return player_info
#http://www.espn.com/nba/player/stats/_/id/2994526/bryn-forbes

In [3]:
player_id_years = find_player_id_per_year([2018])
test_player_info = read_players(player_id_years,[2018])
player_id_years = find_player_id_per_year([2013,2014,2015,2016,2017])
train_player_info = read_players(player_id_years, [2013,2014,2015,2016,2017])

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


{'2017-2018': {'james-harden': '3992', 'anthony-davis': '6583', 'lebron-james': '1966', 'damian-lillard': '6606', 'giannis-antetokounmpo': '3032977', 'stephen-curry': '3975', 'kevin-durant': '3202', 'russell-westbrook': '3468', 'demarcus-cousins': '4258', 'devin-booker': '3136193', 'kyrie-irving': '6442', 'lamarcus-aldridge': '2983', 'victor-oladipo': '2527963', 'demar-derozan': '3978', 'joel-embiid': '3059318', 'kristaps-porzingis': '3102531', 'bradley-beal': '6580', 'lou-williams': '2799', 'jimmy-butler': '6430', 'kemba-walker': '6479', 'paul-george': '4251', 'blake-griffin': '3989', 'cj-mccollum': '2490149', 'karl-anthony-towns': '3136195', 'donovan-mitchell': '3908809', 'khris-middleton': '6609', 'marshon-brooks': '6428', 'klay-thompson': '6475', 't.j.-warren': '2982334', 'john-wall': '4237', 'tyreke-evans': '3983', 'dennis-schroder': '3032979', 'jrue-holiday': '3995', 'harrison-barnes': '6578', 'chris-paul': '2779', 'tobias-harris': '6440', 'nikola-jokic': '3112335', 'eric-gordon'

KeyboardInterrupt: 

In [ ]:
import pickle 
with open('player_data/test_player_info.pkl', 'wb') as fp:
    pickle.dump(test_player_info, fp)
with open('player_data/train_player_info.pkl', 'wb') as fp:
    pickle.dump(train_player_info, fp)

In [ ]:
def player_data_processing(data):
    X = {}
    # print (data[player][2017][1][index:index+2])
    for player in data:
        for year in data[player]:
            feature  = []
            table = np.array(data[player][year][1][0])
            if len(table)<4:
                continue
    #         print (table)
            if 'REGULAR SEASON STATS' not in table:
                continue
            index = int(np.argwhere(table == 'REGULAR SEASON STATS'))
    #         print (index)
            for i in range (1,15):
                x= str(data[player][year][1][index+1:index+2][i])
                x = x.split('\n')[0]
                x = x.split(' ')[-1]
                if '-' in x:
                    x = x.split('-')[-1]
                feature.append(x)
            feature = np.asarray(feature)
            feature = list(map(eval, feature))
            if player not in X:
                X[player]={}
            if year not in X[player]:
                X[player][year] = []
            X[player][year] = feature
    return X

In [ ]:
with open('player_data/test_player_info.pkl', 'rb') as fp:
    test_player_info = pickle.load(fp)
with open('player_data/train_player_info.pkl', 'rb') as fp:
    train_player_info = pickle.load(fp)
    
test_processed_player_data = player_data_processing(test_player_info)
train_processed_player_data = player_data_processing(train_player_info)

with open('player_data/test_processed_player_data.pkl', 'wb') as fp:
    pickle.dump(test_processed_player_data, fp)
with open('player_data/train_processed_player_data.pkl', 'wb') as fp:
    pickle.dump(train_processed_player_data, fp)

In [4]:
print(train_processed_player_data)

{'carmelo-anthony': {'2012-2013': [37.0, 22.2, 0.449, 6.2, 0.379, 7.6, 0.83, 6.9, 2.6, 0.5, 0.8, 3.1, 2.6, 28.7], '2013-2014': [38.7, 21.3, 0.452, 5.4, 0.402, 7.0, 0.848, 8.1, 3.1, 0.7, 1.2, 2.9, 2.6, 27.4], '2014-2015': [35.7, 20.2, 0.444, 4.5, 0.341, 5.9, 0.797, 6.6, 3.1, 0.4, 1.0, 2.2, 2.2, 24.2], '2015-2016': [35.1, 18.2, 0.434, 4.3, 0.339, 5.6, 0.829, 7.7, 4.2, 0.5, 0.9, 2.5, 2.4, 21.8], '2016-2017': [34.3, 18.8, 0.433, 5.7, 0.359, 4.9, 0.833, 5.9, 2.9, 0.4, 0.8, 2.7, 2.1, 22.4]}, 'kevin-durant': {'2012-2013': [38.5, 17.7, 0.51, 4.1, 0.416, 9.3, 0.905, 7.9, 4.6, 1.3, 1.4, 1.8, 3.5, 28.1], '2013-2014': [38.5, 20.8, 0.503, 6.1, 0.391, 9.9, 0.873, 7.4, 5.5, 0.7, 1.3, 2.1, 3.5, 32.0], '2014-2015': [33.8, 17.3, 0.51, 5.9, 0.403, 6.3, 0.854, 6.6, 4.1, 0.9, 0.9, 1.5, 2.7, 25.4], '2015-2016': [35.8, 19.2, 0.505, 6.7, 0.386, 6.9, 0.898, 8.2, 5.0, 1.2, 1.0, 1.9, 3.5, 28.2], '2016-2017': [33.4, 16.5, 0.537, 5.0, 0.375, 6.2, 0.875, 8.3, 4.8, 1.6, 1.1, 1.9, 2.2, 25.1]}, 'kobe-bryant': {'2012-2

In [7]:
print(test_processed_player_data)

{'james-harden': {'2017-2018': [35.4, 20.1, 0.449, 10.0, 0.367, 10.1, 0.858, 5.4, 8.8, 0.7, 1.8, 2.3, 4.4, 30.4]}, 'anthony-davis': {'2017-2018': [36.4, 19.5, 0.534, 2.2, 0.34, 8.0, 0.828, 11.1, 2.3, 2.6, 1.5, 2.1, 2.2, 28.1]}, 'lebron-james': {'2017-2018': [36.9, 19.3, 0.542, 5.0, 0.367, 6.5, 0.731, 8.6, 9.1, 0.9, 1.4, 1.7, 4.2, 27.5]}, 'damian-lillard': {'2017-2018': [36.6, 19.4, 0.439, 8.6, 0.361, 7.4, 0.916, 4.5, 6.6, 0.4, 1.1, 1.6, 2.8, 26.9]}, 'giannis-antetokounmpo': {'2017-2018': [36.7, 18.7, 0.529, 1.9, 0.307, 8.5, 0.76, 10.0, 4.8, 1.4, 1.5, 3.1, 3.0, 26.9]}, 'stephen-curry': {'2017-2018': [32.0, 16.9, 0.495, 9.8, 0.423, 5.9, 0.921, 5.1, 6.1, 0.2, 1.6, 2.2, 3.0, 26.4]}, 'kevin-durant': {'2017-2018': [34.2, 18.0, 0.516, 6.1, 0.419, 5.9, 0.889, 6.8, 5.4, 1.8, 0.7, 2.0, 3.0, 26.4]}, 'russell-westbrook': {'2017-2018': [36.4, 21.1, 0.449, 4.1, 0.298, 7.1, 0.737, 10.1, 10.3, 0.3, 1.8, 2.5, 4.8, 25.4]}, 'demarcus-cousins': {'2017-2018': [36.2, 18.0, 0.47, 6.1, 0.354, 8.2, 0.746, 12.9

# Extract team info

In [395]:
def find_teams_url(team_url):
    html_page = url.urlopen(team_url)
    soup = BeautifulSoup(html_page)
    team_url = []
    for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/team/_/name/")}):
        team_url.append(link.get('href'))
    return set(team_url)


def read_teams(team_url, years):
    team_stat = {}
    for url_path in team_url:
        url_component = url_path.split('/_/')
        stat_url = url_component[0] + '/schedule/_/' + 'name/' + url_component[1].split('/')[1] + '/season/'
        year_stat = {}
        for year in years:
            try:
                dataframe = pandas.read_html(stat_url + str(year) + '/seasontype/2')
                year_stat[year] = dataframe
            except url.HTTPError:
                pass
            print(stat_url + str(year) + '/seasontype/2')
        frames = []
        for year in years:
            df_changed = year_stat[year][2].loc[2:, 0:2]
            before_half = df_changed[0].str.split(" ",expand=True)[1].isin(["Jan","Feb","Mar","Apr","May","Jun","Jul"])
            df_changed[0][before_half] = df_changed[0][before_half]+', '+str(year)
            df_changed[0][~before_half] = df_changed[0][~before_half]+', '+str(year-1)
            df_changed[3] = str(year - 1) + "-" + str(year)
            frames.append(df_changed)
        result = pandas.concat(frames)
        result.index = range(len(result))
        
        team_name_component = url_component[1][5:].split('/')[1].split('-')[:-1]
        team_name = ' '.join(team_name_component)
        
        team_stat[team_name] = result
    return team_stat

def read_teams_total(team_url, years):
    team_total = {}
    for url_path in team_url:
        url_component = url_path.split('/_/')
        stat_url = url_component[0] + '/stats/_/' + 'name/' + url_component[1].split('/')[1] + '/year/'
        year_total = {}
        for year in years:
            try:
                dataframe = pandas.read_html(stat_url + str(year))
                dataframe = pandas.concat([dataframe[0][len(dataframe[0])-1:],dataframe[1][len(dataframe[1])-1:]])
                dataframe.index = range(len(dataframe))
                year_total[year] = dataframe
            except url.HTTPError:
                pass
            print(stat_url + str(year))
        
        team_name_component = url_component[1][5:].split('/')[1].split('-')[:-1]
        team_name = ' '.join(team_name_component)
        
        team_total[team_name] = year_total
    return team_total

In [397]:
team_url = find_teams_url("http://www.espn.com/nba/teams")
test = read_teams(team_url, [2018])
train = read_teams(team_url, [2013,2014,2015,2016,2017])

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://www.espn.com/nba/team/schedule/_/name/lac/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/min/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/det/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/okc/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/chi/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/utah/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bos/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/no/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ny/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/atl/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/phx/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/por/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cle/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_

http://www.espn.com/nba/team/schedule/_/name/wsh/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/wsh/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/wsh/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/wsh/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ind/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ind/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ind/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ind/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/ind/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cha/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cha/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cha/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cha/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/

In [398]:
from datetime import datetime
def data_processing(test):
    game_info = {}
    for team in test:
        for i in range(5,len(test[team][1])):
            #opponent of the current game
            if team not in game_info:
                game_info[team] = []
            #the results of former 5 games of the team
            info = []
            #opponent info
            opp_info = test[team][1][i].strip().lower()
            if opp_info[0] == '@':
                opp = opp_info[1:].strip()
                info.append(opp)
            else:
                opp = opp_info[2:].strip()
                info.append(opp)
            #lose or win
            for j in range(1,6):
                if test[team][2][5-j][0] == 'L':
                    info.append('0')
                else:
                    info.append('1')
            #Home field or away of the current game
            if (test[team][1][i].strip()[0] == 'v'):
                info.append('1')
            else:
                info.append('0')
#             #back to back
#             print(team)
#             d1 = datetime.strptime(test[team][0][i], "%a, %b %d, %Y")
#             d2 = datetime.strptime(test[team][0][i-1], "%a, %b %d, %Y")
#             if (d1-d2).days==1:
#                 info.append('1')
#             else:
#                 info.append('0')
            #the feature vector and result of the current game       
            if (test[team][2][i][0] == 'L'):
                game_info[team].append((info, '0', test[team][3][i]))
            else:
                game_info[team].append((info, '1', test[team][3][i]))
                
    return game_info

In [ ]:
def total_to_dict(data):
    total_dict = {}
    for team in data:
        total_dict[team] = {}
        for year in data[team]:
            lst = [data[team][year][1][0]]
            for i in range(4, 14):
                lst.append(str(data[team][year][i][0]))
            for i in range(1, 15):
                lst.append(str(data[team][year][i][1]))
            total_dict[team][str(year-1) + '-' + str(year)] = lst
    return total_dict

test_total = read_teams_total(team_url, [2018])
train_total = read_teams_total(team_url, [2013,2014,2015,2016,2017])

test_total = total_to_dict(test_total)
train_total = total_to_dict(train_total)

with open('team_data/test_total.pkl', 'wb') as fp:
    pickle.dump(test_total, fp)
with open('team_data/train_total.pkl', 'wb') as fp:
    pickle.dump(train_total, fp)

In [400]:
print(test_total['no/new-orleans-pelicans'])

{'2017-2018': ['82', '111.7', '8.7', '35.7', '44.3', '26.8', '8.01', '5.91', '14.4', '19.1', '1.9', '42.7', '88.3', '.483', '10.2', '28.2', '.362', '16.1', '20.9', '.772', '32.5', '60.1', '.540', '1.27', '0.54']}


In [ ]:
test_total

In [375]:
print(train.keys())

dict_keys(['la', 'minnesota', 'detroit', 'oklahoma city', 'chicago', 'utah', 'boston', 'new orleans', 'new york', 'atlanta', 'phoenix', 'portland trail', 'cleveland', 'philadelphia', 'toronto', 'orlando', 'washington', 'indiana', 'charlotte', 'denver', 'houston', 'dallas', 'brooklyn', 'miami', 'los angeles', 'sacramento', 'golden state', 'san antonio', 'memphis', 'milwaukee'])


In [401]:
print(train['minnesota'])

                     0                 1            2          3
0     Fri, Nov 2, 2012     vs Sacramento       W92-80  2012-2013
1     Sun, Nov 4, 2012         @ Toronto      L105-86  2012-2013
2     Mon, Nov 5, 2012        @ Brooklyn      W107-96  2012-2013
3     Wed, Nov 7, 2012        vs Orlando       W90-75  2012-2013
4     Fri, Nov 9, 2012        vs Indiana       W96-94  2012-2013
5    Sat, Nov 10, 2012         @ Chicago       L87-80  2012-2013
6    Mon, Nov 12, 2012          @ Dallas       W90-82  2012-2013
7    Wed, Nov 14, 2012       vsCharlotte       L89-87  2012-2013
8    Fri, Nov 16, 2012   vs Golden State      L106-98  2012-2013
9    Wed, Nov 21, 2012         vs Denver      L101-94  2012-2013
10   Fri, Nov 23, 2012        @ Portland      L103-95  2012-2013
11   Sat, Nov 24, 2012    @ Golden State       L96-85  2012-2013
12   Tue, Nov 27, 2012      @ Sacramento       W97-89  2012-2013
13   Wed, Nov 28, 2012              @ LA      L101-95  2012-2013
14   Fri, Nov 30, 2012   

In [399]:
import pickle 

all_games_train = data_processing(train)
all_games_test = data_processing(test)

with open('team_data/all_games_train.pkl', 'wb') as fp:
    pickle.dump(all_games_train, fp)
with open('team_data/all_games_test.pkl', 'wb') as fp:
    pickle.dump(all_games_train, fp)

# Extract game info

In [422]:
def read_game_info(team_url, years):
    games_url = set()
    for url_path in team_url:
        url_component = url_path.split('/_/')
        stat_url = url_component[0] + '/schedule/_/' + 'name/' + url_component[1].split('/')[1] + '/season/'
        for year in years:
            try:
                html_page = url.urlopen(stat_url + str(year) + '/seasontype/2')
                soup = BeautifulSoup(html_page)
                for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/game")}):
                    games_url.add("http://www.espn.com/nba/boxscore?gameId=" + link.get('href').split('=')[-1])
            except url.HTTPError:
                pass
    print(games_url)
    games_info = []
    for url_path in games_url:
        try:
            dataframe = pandas.read_html(url_path)
            games_info.append(dataframe)
        except url.HTTPError:
            pass
        
    return games_info

In [423]:
game_info = read_game_info(team_url, [2018])
#example: http://www.espn.com/nba/boxscore?gameId=400974442

http://www.espn.com/nba/team/schedule/_/name/lac/season/2018/seasontype/2


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


http://www.espn.com/nba/game?gameId=400974442
http://www.espn.com/nba/game?gameId=400974788
http://www.espn.com/nba/game?gameId=400974805
http://www.espn.com/nba/game?gameId=400974819
http://www.espn.com/nba/game?gameId=400974835
http://www.espn.com/nba/game?gameId=400974851
http://www.espn.com/nba/game?gameId=400974867
http://www.espn.com/nba/game?gameId=400974882
http://www.espn.com/nba/game?gameId=400974888
http://www.espn.com/nba/game?gameId=400974905
http://www.espn.com/nba/game?gameId=400974923
http://www.espn.com/nba/game?gameId=400974929
http://www.espn.com/nba/game?gameId=400974951
http://www.espn.com/nba/game?gameId=400974971
http://www.espn.com/nba/game?gameId=400974979
http://www.espn.com/nba/game?gameId=400974996
http://www.espn.com/nba/game?gameId=400975007
http://www.espn.com/nba/game?gameId=400975037
http://www.espn.com/nba/game?gameId=400975048
http://www.espn.com/nba/game?gameId=400975068
http://www.espn.com/nba/game?gameId=400975078
http://www.espn.com/nba/game?gameI

http://www.espn.com/nba/game?gameId=400974441
http://www.espn.com/nba/game?gameId=400974787
http://www.espn.com/nba/game?gameId=400974790
http://www.espn.com/nba/game?gameId=400974811
http://www.espn.com/nba/game?gameId=400974825
http://www.espn.com/nba/game?gameId=400974830
http://www.espn.com/nba/game?gameId=400974854
http://www.espn.com/nba/game?gameId=400974880
http://www.espn.com/nba/game?gameId=400974895
http://www.espn.com/nba/game?gameId=400974909
http://www.espn.com/nba/game?gameId=400974919
http://www.espn.com/nba/game?gameId=400974923
http://www.espn.com/nba/game?gameId=400974942
http://www.espn.com/nba/game?gameId=400974963
http://www.espn.com/nba/game?gameId=400974974
http://www.espn.com/nba/game?gameId=400974999
http://www.espn.com/nba/game?gameId=400975014
http://www.espn.com/nba/game?gameId=400975024
http://www.espn.com/nba/game?gameId=400975034
http://www.espn.com/nba/game?gameId=400975055
http://www.espn.com/nba/game?gameId=400975075
http://www.espn.com/nba/game?gameI

http://www.espn.com/nba/game?gameId=400974437
http://www.espn.com/nba/game?gameId=400974703
http://www.espn.com/nba/game?gameId=400974772
http://www.espn.com/nba/game?gameId=400974802
http://www.espn.com/nba/game?gameId=400974818
http://www.espn.com/nba/game?gameId=400974829
http://www.espn.com/nba/game?gameId=400974843
http://www.espn.com/nba/game?gameId=400974860
http://www.espn.com/nba/game?gameId=400974880
http://www.espn.com/nba/game?gameId=400974889
http://www.espn.com/nba/game?gameId=400974897
http://www.espn.com/nba/game?gameId=400974912
http://www.espn.com/nba/game?gameId=400974921
http://www.espn.com/nba/game?gameId=400974939
http://www.espn.com/nba/game?gameId=400974952
http://www.espn.com/nba/game?gameId=400974966
http://www.espn.com/nba/game?gameId=400974981
http://www.espn.com/nba/game?gameId=400975000
http://www.espn.com/nba/game?gameId=400975008
http://www.espn.com/nba/game?gameId=400975020
http://www.espn.com/nba/game?gameId=400975032
http://www.espn.com/nba/game?gameI

...

http://www.espn.com/nba/game?gameId=400974702
http://www.espn.com/nba/game?gameId=400974774
http://www.espn.com/nba/game?gameId=400974779
http://www.espn.com/nba/game?gameId=400974801
http://www.espn.com/nba/game?gameId=400974822
http://www.espn.com/nba/game?gameId=400974839
http://www.espn.com/nba/game?gameId=400974848
http://www.espn.com/nba/game?gameId=400974863
http://www.espn.com/nba/game?gameId=400974871
http://www.espn.com/nba/game?gameId=400974889
http://www.espn.com/nba/game?gameId=400974911
http://www.espn.com/nba/game?gameId=400974925
http://www.espn.com/nba/game?gameId=400974936
http://www.espn.com/nba/game?gameId=400974950
http://www.espn.com/nba/game?gameId=400974964
http://www.espn.com/nba/game?gameId=400974980
http://www.espn.com/nba/game?gameId=400974994
http://www.espn.com/nba/game?gameId=400975012
http://www.espn.com/nba/game?gameId=400975020
http://www.espn.com/nba/game?gameId=400975029
http://www.espn.com/nba/game?gameId=400975041
http://www.espn.com/nba/game?gameI

http://www.espn.com/nba/game?gameId=400974700
http://www.espn.com/nba/game?gameId=400974770
http://www.espn.com/nba/game?gameId=400974795
http://www.espn.com/nba/game?gameId=400974806
http://www.espn.com/nba/game?gameId=400974821
http://www.espn.com/nba/game?gameId=400974839
http://www.espn.com/nba/game?gameId=400974847
http://www.espn.com/nba/game?gameId=400974856
http://www.espn.com/nba/game?gameId=400974877
http://www.espn.com/nba/game?gameId=400974894
http://www.espn.com/nba/game?gameId=400974903
http://www.espn.com/nba/game?gameId=400974921
http://www.espn.com/nba/game?gameId=400974958
http://www.espn.com/nba/game?gameId=400974973
http://www.espn.com/nba/game?gameId=400974979
http://www.espn.com/nba/game?gameId=400974992
http://www.espn.com/nba/game?gameId=400975004
http://www.espn.com/nba/game?gameId=400975021
http://www.espn.com/nba/game?gameId=400975028
http://www.espn.com/nba/game?gameId=400975058
http://www.espn.com/nba/game?gameId=400975072
http://www.espn.com/nba/game?gameI

http://www.espn.com/nba/game?gameId=400974705
http://www.espn.com/nba/game?gameId=400974776
http://www.espn.com/nba/game?gameId=400974783
http://www.espn.com/nba/game?gameId=400974796
http://www.espn.com/nba/game?gameId=400974812
http://www.espn.com/nba/game?gameId=400974817
http://www.espn.com/nba/game?gameId=400974832
http://www.espn.com/nba/game?gameId=400974849
http://www.espn.com/nba/game?gameId=400974867
http://www.espn.com/nba/game?gameId=400974876
http://www.espn.com/nba/game?gameId=400974885
http://www.espn.com/nba/game?gameId=400974902
http://www.espn.com/nba/game?gameId=400974932
http://www.espn.com/nba/game?gameId=400974942
http://www.espn.com/nba/game?gameId=400974954
http://www.espn.com/nba/game?gameId=400974975
http://www.espn.com/nba/game?gameId=400974984
http://www.espn.com/nba/game?gameId=400975000
http://www.espn.com/nba/game?gameId=400975011
http://www.espn.com/nba/game?gameId=400975034
http://www.espn.com/nba/game?gameId=400975046
http://www.espn.com/nba/game?gameI

http://www.espn.com/nba/game?gameId=400974442
http://www.espn.com/nba/game?gameId=400974777
http://www.espn.com/nba/game?gameId=400974791
http://www.espn.com/nba/game?gameId=400974815
http://www.espn.com/nba/game?gameId=400974827
http://www.espn.com/nba/game?gameId=400974833
http://www.espn.com/nba/game?gameId=400974855
http://www.espn.com/nba/game?gameId=400974869
http://www.espn.com/nba/game?gameId=400974881
http://www.espn.com/nba/game?gameId=400974896
http://www.espn.com/nba/game?gameId=400974912
http://www.espn.com/nba/game?gameId=400974915
http://www.espn.com/nba/game?gameId=400974933
http://www.espn.com/nba/game?gameId=400974947
http://www.espn.com/nba/game?gameId=400974965
http://www.espn.com/nba/game?gameId=400974978
http://www.espn.com/nba/game?gameId=400974991
http://www.espn.com/nba/game?gameId=400975003
http://www.espn.com/nba/game?gameId=400975017
http://www.espn.com/nba/game?gameId=400975048
http://www.espn.com/nba/game?gameId=400975063
http://www.espn.com/nba/game?gameI

http://www.espn.com/nba/game?gameId=400974440
http://www.espn.com/nba/game?gameId=400974782
http://www.espn.com/nba/game?gameId=400974797
http://www.espn.com/nba/game?gameId=400974810
http://www.espn.com/nba/game?gameId=400974822
http://www.espn.com/nba/game?gameId=400974837
http://www.espn.com/nba/game?gameId=400974843
http://www.espn.com/nba/game?gameId=400974868
http://www.espn.com/nba/game?gameId=400974877
http://www.espn.com/nba/game?gameId=400974892
http://www.espn.com/nba/game?gameId=400974905
http://www.espn.com/nba/game?gameId=400974924
http://www.espn.com/nba/game?gameId=400974934
http://www.espn.com/nba/game?gameId=400974954
http://www.espn.com/nba/game?gameId=400974961
http://www.espn.com/nba/game?gameId=400974974
http://www.espn.com/nba/game?gameId=400975001
http://www.espn.com/nba/game?gameId=400975013
http://www.espn.com/nba/game?gameId=400975028
http://www.espn.com/nba/game?gameId=400975046
http://www.espn.com/nba/game?gameId=400975062
http://www.espn.com/nba/game?gameI

In [424]:
print(len(game_info))

1226


In [432]:
print(len(game_info[0]))

5
